# Get Libraries

In [20]:
# Get some libraries
import pandas as pd
import numpy as np
import requests

import plotly.plotly as py
import plotly.tools as tools
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as py_off
from plotly.graph_objs import *

Set username for mapbox

In [21]:
# Fill these Variables with your copy

# Mapbox Access Token
# Will need to create a free Mapbox account to use
my_mapbox_access_token = 'pk.eyJ1IjoiamJvbmlmaWVsZDMiLCJhIjoiY2o5bjJrdjRjNHl4NTMzcW0zanR4Z2Z6eCJ9.ooDfIgzxDtQTHq5GPHBErA'

# Plotly Username and API Key
# Will need to create a free Plot.ly account to use

username = 'dcarvallo3'
plotly_api_key = 'wjACCAYB0hYBeMemhcJW'

### Get local CSV data

get local CSV data, apply latitude/longitdue flip and trim data to continental USA. 

In [22]:

df = pd.read_csv(r'C:\Users\dcarv\Downloads\CO2-data.csv')
data = []
#plot one year
year = 2013
df_set_yr = df[['LAT', 'LONG', str(year)]]
df_set_yr["DLONG"] = df['LONG'].apply(lambda x: x-180 if x<180 else x-180)
df_set_yr["DLAT"] = df['LAT'].apply(lambda x: -x+90 if x<90 else -x+90)
df_set_yr = df_set_yr[df_set_yr["DLONG"] < -60]
df_set_yr = df_set_yr[df_set_yr["DLONG"] > -126]
df_set_yr = df_set_yr[df_set_yr["DLAT"] > 15]
df_set_yr = df_set_yr[df_set_yr["DLAT"] < 50]

C:\Users\dcarv\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\dcarv\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Interpolate all data to a given resolution


In [23]:
#create a finer mesh
inter_num=10
from scipy.interpolate import interp1d, interp2d, interpnd
x, y, z  = df_set_yr["DLONG"].values, df_set_yr["DLAT"].values, df_set_yr[str(year)].values
# Create the function Z_inter for interpolation:
Z_inter = interp2d(x, y, z)

# Create the x_mesh and y_mesh :
x_mesh = np.linspace(x.min(), x.max(), num=len(set(x))*inter_num)
y_mesh = np.linspace(y.min(), y.max(), num=len(set(y))*inter_num)

#print(x_mesh)
#print(y_mesh)
Z_interpolated = Z_inter(x_mesh, y_mesh)
print(Z_interpolated.shape)
#values=list(Z_interpolated.flatten())

LATS=[]
LONG=[]
VALS=[]
for xn in range(Z_interpolated.shape[1]):
    for yn in range(Z_interpolated.shape[0]):
        LATS.append(y_mesh[yn])
        LONG.append(x_mesh[xn])
        VALS.append(Z_interpolated[yn,xn])
        
        
#print(len(LATS))
assert len(LATS)==len(VALS)
assert len(LONG)==len(VALS)
assert len(LATS)==len(LONG)
interp_df = pd.DataFrame(list(zip(LATS, LONG, VALS)),
              columns=['LAT','LONG', 'VAL'])
interp_df = interp_df[interp_df['VAL']>0.05]


(340, 650)


In [24]:
print(interp_df.shape)

(101817, 3)


In [25]:
# Color Scale
scl = [0,"rgb(255,255,255)"],[0.25,"rgb(0, 0, 200)"],[0.375,"rgb(0, 25, 255)"],\
[0.375,"rgb(0, 152, 255)"],[0.5,"rgb(44, 255, 150)"],[0.625,"rgb(151, 255, 0)"],\
[0.75,"rgb(255, 234, 0)"],[0.875,"rgb(255, 111, 0)"],[1,"rgb(255, 0, 0)"]

Plot the final result

In [26]:
data=[]
data.append(
    go.Scattermapbox(
        lon = interp_df['LONG'],#.apply(lambda x: x-180 if x<180 else x-180),
        lat = interp_df['LAT'],#.apply(lambda x: -x+90 if x<90 else -x+90),
        mode='markers',
        marker=Marker(
            color=interp_df['VAL'].values,
            colorscale='Jet'
        ),
    )
)

layout = Layout(
    margin=dict(t=0,b=0,r=0,l=0),
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=my_mapbox_access_token,
        bearing=0,
        center=dict(
            lat=38,
            lon=-94
        ),
        pitch=0,
        zoom=0,
        style='light'
    ),
)
 
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='CO2 data in '+str(year))